In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import RANSACRegressor

import seaborn as sns
import xgboost as xgb
import lightgbm as lgb
import catboost as cb
import tensorflow as tf
import keras
from keras import layers

In [2]:
df = pd.read_csv('dataset_regression/parkinsons_updrs_cleaned.data')

In [3]:
df.head()

,age,sex,motor_UPDRS,Jitter_pct,Jitter_Abs,Jitter_RAP,Jitter_PPQ5,Jitter_DDP,Shimmer,Shimmer_dB,Shimmer_APQ3,Shimmer_APQ5,Shimmer_APQ11,Shimmer_DDA,NHR,HNR,RPDE,DFA,PPE
0,72,0,28.199,0.00662,0.000034,0.00401,0.00317,0.01204,0.02565,0.230,0.01438,0.01309,0.01662,0.04314,0.014290,21.640,0.41888,0.54842,0.16006
1,72,0,28.447,0.00300,0.000017,0.00132,0.00150,0.00395,0.02024,0.179,0.00994,0.01072,0.01689,0.02982,0.011112,27.183,0.43493,0.56477,0.10810
2,72,0,28.695,0.00481,0.000025,0.00205,0.00208,0.00616,0.01675,0.181,0.00734,0.00844,0.01458,0.02202,0.020220,23.047,0.46222,0.54405,0.21014
3,72,0,28.905,0.00528,0.000027,0.00191,0.00264,0.00573,0.02309,0.327,0.01106,0.01265,0.01963,0.03317,0.027837,24.445,0.48730,0.57794,0.33277
4,72,0,29.187,0.00335,0.000020,0.00093,0.00130,0.00278,0.01703,0.176,0.00679,0.00929,0.01819,0.02036,0.011625,26.126,0.47188,0.56122,0.19361


In [4]:
df.shape

(5070, 19)

In [5]:
X = df.drop(['motor_UPDRS'], axis=1)
y = df['motor_UPDRS']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)   

In [7]:
print("Overall distribution:")
df['motor_UPDRS'].describe()    

Overall distribution:


count    5070.000000
mean       21.436351
std         8.197695
min         5.037700
25%        14.863000
50%        21.300000
75%        27.793000
max        39.511000
Name: motor_UPDRS, dtype: float64

In [8]:

print("\nTraining set distribution:")
y_train.describe()  



Training set distribution:


count    4056.000000
mean       21.463270
std         8.188898
min         5.037700
25%        14.863000
50%        21.372000
75%        27.793000
max        39.511000
Name: motor_UPDRS, dtype: float64

In [9]:

print("\nTest set distribution:")
y_test.describe()   



Test set distribution:


count    1014.000000
mean       21.328674
std         8.235962
min         5.037700
25%        14.839000
50%        20.855000
75%        27.793000
max        39.511000
Name: motor_UPDRS, dtype: float64

In [10]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [11]:
# Creating a dictionary of models
models = {
    "Linear Regression": LinearRegression(),
    "Random Forest Regressor": RandomForestRegressor(),
    "Support Vector Regressor": SVR(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "MLP Regressor": MLPRegressor(),
    "Decision Tree Regressor": DecisionTreeRegressor(),
    "Extra Trees Regressor": ExtraTreesRegressor(),
    "Gradient Boosting Regressor": GradientBoostingRegressor(),
    "XGBoost": xgb.XGBRegressor(),
    "LightGBM": lgb.LGBMRegressor(),
    "CatBoost": cb.CatBoostRegressor(verbose=0),
    "RANSAC Regressor": RANSACRegressor(),
    "Neural Network": keras.models.Sequential([
        layers.Dense(128, activation='relu', input_shape=[X_train.shape[1]]),
        layers.Dense(64, activation='relu'),
        layers.Dense(32, activation='relu'),
        layers.Dense(1)
    ])
}

c:\Users\murta\Desktop\Desktop\ML&DE\Fourth_Semester\Advanced_ML\.venv\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [12]:
# Creating a list to store the results
results = []
# for each model
for name, model in models.items():
    # compile the Keras model if it's a neural network
    if name == "Neural Network":
        model.compile(optimizer='adam', loss='mean_squared_error')
    # fit the model
    model.fit(X_train_scaled, y_train)
    # predict the test set
    y_pred = model.predict(X_test_scaled)
    # calculate the metrics
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    # append the results to the list
    results.append([name, mae, mse, r2])
    

c:\Users\murta\Desktop\Desktop\ML&DE\Fourth_Semester\Advanced_ML\.venv\lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\murta\Desktop\Desktop\ML&DE\Fourth_Semester\Advanced_ML\.venv\lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
  File "C:\Users\murta\AppData\Local\Programs\Python\Python310\lib\subprocess.py", line 503, in run
    with Popen(*popenargs, **kwargs) as process:
  File "C:\Users\murta\AppData\Local\Programs\Python\Python310\lib\subprocess.py", line 971, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "C:\Users\murta\AppData\Local\Pr

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001219 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4107
[LightGBM] [Info] Number of data points in the train set: 4056, number of used features: 18
[LightGBM] [Info] Start training from score 21.463270
127/127 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 335.0524
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


In [13]:
# Create a DataFrame from the results list
results_df = pd.DataFrame(results, columns=['Model', 'MAE', 'MSE', 'R2'])
results_df

,Model,MAE,MSE,R2
0,Linear Regression,5.911461,50.435082,0.255726
1,Random Forest Regressor,1.834991,6.381479,0.905828
2,Support Vector Regressor,4.975825,40.269304,0.405743
3,K-Neighbors Regressor,3.295753,22.629438,0.666056
4,MLP Regressor,4.266551,28.817201,0.574743
5,Decision Tree Regressor,2.188353,11.970916,0.823344
6,Extra Trees Regressor,1.805815,6.171029,0.908934
7,Gradient Boosting Regressor,3.112458,15.651731,0.769026
8,XGBoost,2.046603,7.503314,0.889273
9,LightGBM,1.934148,6.756447,0.900295
